In [26]:
#Proof of Concept

In [27]:
#Imports
import pandas as pd
import os
import pathlib
import shutil
import data_analysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from joblib import dump, load
import statistics
import numpy as np

In [28]:
#Paths
path = pathlib.Path().cwd().parent
path_labeled = path / "Data" / "Combined"
path_train = path / "Data" / "Train"
path_validation = path / "Data" / "Validation"

path_train.mkdir(parents=True, exist_ok=True)
path_validation.mkdir(parents=True, exist_ok=True)

train_data = [file.name for file in path_train.iterdir()]
train_file_paths = [(path_train / file) for file in train_data]

validation_data = [file.name for file in path_validation.iterdir()]
validation_file_paths = [(path_validation / file) for file in validation_data]

#Activity labels
Activities = ["BHD", "FHD", "SRV", "VOL"]

#Activity mapping function
def act(activity: str):
    for i in range(len(Activities)):
        if activity == Activities[i]:
            return i
    return -1


In [29]:
#Create validation and training sets
#First 5 is validation, Last 25 is training
files_in_data = [file.name for file in path_labeled.iterdir()]
for file in files_in_data:
    if file[0:3] in Activities: 
        if int(file[4:-4]) <= 5:
            shutil.copy(path_labeled / file, path_validation)
        else:
            shutil.copy(path_labeled / file, path_train)

In [30]:
#Activity mapping function
def act(activity: str):
    for i in range(len(Activities)):
        if activity == Activities[i]:
            return i
    return -1


#Generates features from csv
def summarize(csv_file: str):
    
    data = pd.read_csv(csv_file, index_col=False)
    start, end = data_analysis.get_start_end(data)
    data = data_analysis.add_features(data)
    data = data[start:end]
    result = [0]*80
    for index, col in enumerate(data):
        if col != "time":
            if(index > 40):
                break
            result[2*(index-1)], result[2*(index-1)+1] = data[col].mean(), data[col].var()

    return result

#Generates features/label sets of data from file paths
def preprocess(filepathset):
    features = [0] * 2
    features[0] = [summarize(file) for file in filepathset] #summarization
    features[1] = [act(file.name[0:3]) for file in filepathset] #label
    return features

#Takes x training samples from each person in filepath set.
def xsamples(filepathset, samples):
    return [file for file in filepathset if int(file.name[-6:-4]) < ( samples + 6)]

In [31]:
samples = 25
train_features = preprocess(xsamples(train_file_paths,samples))


In [32]:
model = RandomForestClassifier()
model.fit(train_features[0], train_features[1])
y_pred = model.predict(train_features[0])
accuracy = accuracy_score(train_features[1], y_pred)
print("accuracy (test) :", accuracy)

#validation
validation_features = preprocess(validation_file_paths)
y_pred_2 = model.predict(validation_features[0])
accuracy2 = accuracy_score(validation_features[1], y_pred_2)
print("accuracy (validation) :", accuracy2)

accuracy (test) : 1.0
accuracy (validation) : 1.0
